In [1]:
import numpy as np
from sgfmill import sgf
import pandas as pd

import Go_Board

from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, ZeroPadding2D


from os import listdir
from os.path import isfile, join

In [2]:
dataset_dir = './go9-large/'
board_size = 9
proportion_dataset = 45000
training = False
data_preprocessing = True

In [3]:
if data_preprocessing:
    examples = [f for f in listdir(dataset_dir) if isfile(join(dataset_dir, f))]
    print(len(examples),"Games")
    examples = examples[:proportion_dataset]

49894 Games


In [4]:
X = []
Y = []
if data_preprocessing:
    for example in examples:
        board_go = Go_Board.Our_Board(board_size=board_size)
        with open(dataset_dir+example, "rb") as f:
            game = sgf.Sgf_game.from_bytes(f.read())
        winner = game.get_winner()
        board_size = game.get_size()
        root_node = game.get_root()

        board = np.zeros(shape=(board_size,board_size))

        for node in game.get_main_sequence():

            if node.get_move()[0] is not None:
                move = node.get_move()
                player = move[0]
                coord = (move[1][1],move[1][0]) if move[1] is not None else (-1,-1)

                board  = board_go.get_matrix()

                y = np.zeros(shape=(board_size,board_size))
                if(coord[0] != -1 and coord[1] != -1):
                    y[coord[0]][coord[1]] = 1
                    y = y.reshape(board_size*board_size)
                    y = np.append(y,[0])
                    y = y.reshape(board_size*board_size+1)
                else:
                    y = np.append(y,[1])
                    y = y.reshape(board_size*board_size+1)

                x = board * -1 if player == 'w' else board
                board_go.push(coord)
                X.append(x)
                Y.append(y)

In [5]:
if data_preprocessing:
    X = np.reshape(X,(len(X),board_size,board_size,1))
    print("X shape = ",X.shape)

    Y = np.reshape(Y,(len(Y),82))
    print("Y shape = ",Y.shape)

X shape =  (2166424, 9, 9, 1)
Y shape =  (2166424, 82)


In [6]:
if data_preprocessing:
    np.save("dataset_X", X)
    np.save("dataset_Y", Y)
else:
    X = np.load('dataset_X.npy')
    Y = np.load("dataset_Y.npy")
    print("X shape = ",X.shape)
    print("Y shape = ",Y.shape)

OSError: Not enough free space to write 1421174144 bytes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

In [13]:
model = models.Sequential()

model.add(ZeroPadding2D(padding=3, input_shape=(9,9,1), data_format='channels_first'))
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2, data_format='channels_first'))
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2, data_format='channels_first'))
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
model.add(Activation('relu'))

model.add(ZeroPadding2D(padding=2, data_format='channels_first'))
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
model.add(Activation('relu'))

model.add(layers.Flatten())

model.add(layers.Dense(82, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 9, 15, 7)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 13, 5)         2624      
_________________________________________________________________
activation (Activation)      (None, 32, 13, 5)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 32, 17, 9)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 15, 7)         9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 15, 7)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 32, 19, 11)        0

In [15]:
if training:
    SGD = keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)
    Adam = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=SGD,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=5,batch_size=128)


Train on 806884 samples, validate on 397421 samples
Epoch 1/5
500736/806884 [=================>............] - ETA: 6:55 - loss: 4.1347 - accuracy: 0.0359

KeyboardInterrupt: 

In [24]:
if training:
    model.save('models/model_'+str(board_size)+'_'+str(proportion_dataset)+'.h5')

In [ ]:
new_model = keras.models.load_model('models/model_'+str(board_size)+'_'+str(proportion_dataset)+'.h5')

In [ ]:
pred = new_model.predict(X_test)

In [ ]:
def get_coord(pred):
    return pred//board_size,pred%board_size

In [ ]:
print(X_test[0].reshape(9,9))
print(get_coord(np.argmax(pred[0])))